# A Slighly More Complex Problem

Let's create now a capacitor problem in which we prescribe the potential at the electrodes. The equation that we need to solve is the following:

Find $u$ such that
$$
\begin{align*} -\varepsilon \Delta u &= 0 \quad \text{in } \Omega, \\ u &= 1 \quad \text{on } \text{Electrode 1},\\ u &= -1 \quad \text{on } \text{Electrode 2},\\ n\cdot \nabla u &= 0 \quad \text{on } \text{outer}, \end{align*}
$$
With permettivity given by:
$$
\begin{align*} \varepsilon = \begin{cases} 1 & \text{in air},\\ 2 & \text{in dielectric}. \end{cases} \end{align*}
$$
And domain as in figure below:
<div style="text-align:center;">
<svg width="400" height="400" viewBox="-2.1 -2.1 4.2 4.2">
    <!-- Circle from (0, 0) with radious 2 and call it "air" with black border -->
    <circle cx="0" cy="0" r="2" fill="white" stroke="black" stroke-width="0.01"/>
    <!-- Electrodes -->
    <rect x="-0.4" y="0.2" width="0.8" height="0.1" fill="black"/>
    <rect x="-0.4" y="-0.2" width="0.8" height="0.1" fill="black"/>
    <!-- Dielectric -->
    <rect x="-0.9" y="-0.1" width="1.8" height="0.3" fill="gray"/>
    <!--Add the names air Electrode 1 Electrode 2 Dielectric -->
    <text x="-0.5" y="0.7" font-size="0.09" fill="black">Air</text>
    <text x="-0.4" y="0.3" font-size="0.09" fill="white">Electrode</text>
    <text x="-0.9" y="0.1" font-size="0.09" fill="black">Dielectric</text>
</svg>
</div>



In [1]:
from ngsolve import *
from ngsolve.webgui import Draw
from netgen.occ import *

In [2]:
square = Circle( (0,0), 2).Face() # create a rectangle with boundary conditions
square.edges.name = "outer"
square.faces.name = "air"

el1 = MoveTo(-0.4, 0.2).Rectangle( 0.8, 0.1).Face() # create a rectangle with boundary conditions
el1.edges.name = "el1"
el1.vertices.name = "el1"

el2 = MoveTo(-0.4, -0.2).Rectangle( 0.8, 0.1).Face() # create a rectangle with boundary conditions
el2.edges.name = "el2"
el2.vertices.name = "el2"

dielec = MoveTo(-0.9, -0.1).Rectangle( 1.8, 0.3).Face()
dielec.faces.name = "dielec"

air = square  # subtract the rectangles from the air rectangle
shape = Glue([air - dielec, dielec])
shape =shape - el1 - el2

### adding extra specifications on the shape
#predefined mesh size for the shape

shape.edges["el.*"].maxh = 0.1 # set the mesh size of the edges with the name "el.*" to 0.05
shape.vertices["el.*"].maxh = 0.05 # set the mesh size of the vertices with the name "el.*" to 0.05


Draw(shape); # draw the shape


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': …

In [3]:

mesh = Mesh(OCCGeometry(shape, dim = 2).GenerateMesh(maxh=0.5)).Curve(4) # create a mesh from the shape

eps = CoefficientFunction( [1, 2] ) # define the coefficient function # try sin(5*x) instead of 2 

Draw(eps, mesh, "eps"); # draw the coefficient function


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [4]:
help(mesh)

Help on Mesh in module ngsolve.comp object:

class Mesh(pybind11_builtins.pybind11_object)
 |  NGSolve interface to the Netgen mesh. Provides access and functionality
 |  to use the mesh for finite element calculations.
 |  
 |  Parameters:
 |  
 |  mesh (netgen.Mesh): a mesh generated from Netgen
 |  
 |  Method resolution order:
 |      Mesh
 |      pybind11_builtins.pybind11_object
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  BBBoundaries(...)
 |      BBBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 3 boundary mesh-region matching the given regex pattern
 |  
 |  BBoundaries(...)
 |      BBoundaries(self: ngsolve.comp.Mesh, pattern: str) -> ngsolve.comp.Region
 |      
 |      Return co dim 2 boundary mesh-region matching the given regex pattern
 |  
 |  Boundaries(...)
 |      Boundaries(*args, **kwargs)
 |      Overloaded function.
 |      
 |      1. Boundaries(self: ngsolve.comp.Mesh, pattern: str) -> ng

The bilinear and liner forms now are :
\begin{align*} a(u,v) &= \int_{\Omega} \varepsilon \nabla u \cdot \nabla v \, dx,\\ f(v) &= 0. \end{align*}

In [5]:
order = 3
fes = H1(mesh, order=order, dirichlet="el.*") # define the finite element space

u ,v = fes.TnT() # define the trial and test functions

a = BilinearForm(fes)
a += eps * grad(u) * grad(v) * dx
a.Assemble()

f = LinearForm(fes)
f.Assemble() # not needed it is zero

We dont have homogeneous Dirichlet boundary conditions, so we need a homogenization technique.

Homogenization technique in a nutshell:
Suppose we the solution $u_{\text{sol}} = u_0 + u_{D}$, where $u_0$ is the solution of the problem with homogeneous boundary conditions and $ u_{D}$ is the solution 

\begin{align*}
A(u_{\text{sol}}, v ) = f(v)\\
A(u_0 + u_D, v ) = f(v)\\
A(u_0, v )  = f(v)-A(u_D, v )\\
\end{align*}

After the discretization is equivalent to solve the following linear system:
\begin{align*}
u_0 = A^{-1}(f-Au_D)
\end{align*}
The solution can be found as
\begin{align*}
u_\text{sol} = A^{-1}(f-Au_{D}) + u_D
\end{align*}

In [6]:

gfu = GridFunction(fes) # define the grid function
electrode = mesh.BoundaryCF({"el1":1, "el2":-1}, default = 0 ) # define the boundary conditions
gfu.Set(electrode, definedon=mesh.Boundaries("el.*")) # set the boundary conditions

Draw(gfu, deformation =True); # draw the grid function


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [7]:

gfu.vec.data += a.mat.Inverse(freedofs = fes.FreeDofs()) * (f.vec - a.mat * gfu.vec) # solve the system
Draw(gfu , deformation =True, scale = 1); # draw the solution


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

In [8]:

# Electic field
Draw(-eps*grad(gfu) , mesh, "E", min = 0, max = 4); # draw the solution

# if you zoom in you will see the difference in the two materials.

WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.2…

### Some visualization tools

In [9]:
from ngsolve.webgui import Draw, FieldLines, AddFieldLines
fes_flux = HDiv(mesh, order=order-1) # Same as 
#fes_flux = H1(mesh, order=order-1, dim=mesh.dim) 
# fes_flux = HDiv(mesh, order=order-1)


flux = GridFunction(fes_flux, name="flux")
flux.Set( eps*grad(gfu) )

N = 10
#fac = 0 if mesh.dim == 2 else 1
#p = [(-1+4*i/N,-2+4*j/N,fac * 2*k/N) for i in range(1,2*N) for j in range(1,2*N) for k in range(1,N)]
p = [(i / N, -1 + (2*j)/ N, 0) for i in range(N+1) for j in range(N+1) ] 

fieldlines = flux._BuildFieldLines(mesh, p, num_fieldlines=N**3//5, randomized=False, length=0.3)

clipping = { "clipping" : { "y":0, "z":-1} }


Draw(-eps*grad(gfu), mesh,  "X", draw_vol=True, draw_surf=True, objects=[fieldlines], \
     autoscale=True, min = 0, max = 10, settings={"Objects": {"Surface": False}}, **clipping);


WebGuiWidget(layout=Layout(height='500px', width='100%'), value={'gui_settings': {'Objects': {'Surface': False…